In [28]:
import pandas as pd

df=pd.read_csv('merged_training_table.csv')
df.head()

,month,sector,year,amount_new_house_transactions,year_end_registered_population_10k,total_households_10k,year_end_resident_population_10k,year_end_total_employed_population_10k,year_end_urban_non_private_employees_10k,private_individual_and_other_employees_10k,...,medical_health_rehabilitation_institution_dense,medical_health_first_aid_center_dense,medical_health_blood_donation_station_dense,medical_health_disease_prevention_institution_dense,medical_health_general_hospital_dense,medical_health_clinic_dense,education_training_school_education_middle_school_dense,education_training_school_education_primary_school_dense,education_training_school_education_kindergarten_dense,education_training_school_education_research_institution_dense
0,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014
1,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014
2,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014
3,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014
4,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014


In [29]:
X=df.drop('amount_new_house_transactions',axis=1)
y=df['amount_new_house_transactions']

non_numeric = X.select_dtypes(exclude=["number"]).columns.tolist()
print("非數值欄位:", non_numeric)


非數值欄位: ['month', 'sector', 'keyword', 'source']


In [30]:
df['month'].dtypes

dtype('O')

In [31]:
df['month'] = pd.to_datetime(df['month'])
df['year'] = df['year'].astype('category')

In [32]:
X = df.drop('amount_new_house_transactions', axis=1)  # 轉型後再建
non_numeric = X.select_dtypes(exclude=['number']).columns.tolist()
print("非數值欄位:", non_numeric)

非數值欄位: ['month', 'sector', 'year', 'keyword', 'source']


In [33]:
df['month'].dtypes

dtype('<M8[ns]')

In [34]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [35]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit , cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np

num_cols=X.select_dtypes(include=['number']).columns.tolist()
cat_cols=X.select_dtypes(exclude=['number']).columns.tolist()

pre_linear=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
        ]),num_cols),
    ]
)

pre_tree=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
        ]),num_cols),
    ]
)


linear=LinearRegression()
pipe_linear=Pipeline([
    ('pre_linear',pre_linear),
    ('linear',linear),
])

rf=RandomForestRegressor(
    n_estimators=100,        
    max_depth=12,            
    min_samples_split=20,    
    min_samples_leaf=5,      
    max_features='sqrt',   
    bootstrap=True,          
    max_samples=0.7,         
    n_jobs=-1,
    random_state=42,
)
pipe_rf=Pipeline([
    ('pre_rf',pre_tree),
    ('rf',rf),
])

xgb=XGBRegressor(
    n_estimators=200,        
    max_depth=6,             
    learning_rate=0.1,      
    subsample=0.8,           
    colsample_bytree=0.8,    
    min_child_weight=5,      
    reg_lambda=1.0,          
    n_jobs=-1,              
    random_state=42,
    tree_method="hist"            
)
pipe_xgb=Pipeline([
    ('pre_xgb',pre_tree),
    ('xgb',xgb),
])




#設定評分方法
# —— 兩階段 MAPE：回傳「分數，越大越好」——
def two_stage_mape_score(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)

    # APE：處理 y_true=0 的情況（與常識一致：真值為 0 且預測也為 0 → 0；否則 → inf）
    ape = np.where(
        y_true == 0,
        np.where(np.isclose(y_pred, 0.0), 0.0, np.inf),
        np.abs(y_pred - y_true) / np.abs(y_true)
    )

    n = len(ape)
    frac_big = np.mean(ape > 1)  # >100% 的比例

    # 第一階段：若 >30% 的樣本 APE>1 → 直接 0 分
    if frac_big > 0.3:
        return 0.0

    # 第二階段：只用 APE<=1 的樣本，計算縮放後的 MAPE，score = 1 - scaled_mape
    mask_ok = (ape <= 1) & np.isfinite(ape)
    if mask_ok.sum() == 0:
        return 0.0  # 理論上不會到這裡，保險起見

    mape_ok = ape[mask_ok].mean()
    frac_ok = mask_ok.sum() / n
    scaled_mape = mape_ok / frac_ok
    score = 1.0 - scaled_mape

    # 分數界於 [0,1]（理論上已是如此，這裡穩健處理）
    return float(np.clip(score, 0.0, 1.0))

# 包成 sklearn scorer（越大越好）
kaggle_scorer = make_scorer(two_stage_mape_score, greater_is_better=True)


tscv=TimeSeriesSplit(n_splits=5)

cv_scores_linear=cross_val_score(
    pipe_linear,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("Linear:CV (Kaggle two-stage score) mean:", cv_scores_linear.mean())
print("Linear:Each split:", cv_scores_linear)

cv_scores_rf=cross_val_score(
    pipe_rf,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("RF:CV (Kaggle two-stage score) mean:", cv_scores_rf.mean())
print("RF:Each split:", cv_scores_rf)

cv_scores_xgb=cross_val_score(
    pipe_xgb,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("XGB:CV (Kaggle two-stage score) mean:", cv_scores_xgb.mean())
print("XGB:Each split:", cv_scores_xgb)

c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\USER\AppData\Local\Temp\ipykernel_23076\71088798.py:85: RuntimeWarning: divide by zero encountered in divide
  np.abs(y_pred - y_true) / np.abs(y_true)
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages

Linear:CV (Kaggle two-stage score) mean: 0.0
Linear:Each split: [0. 0. 0. 0. 0.]


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\USER\AppData\Local\Temp\ipykernel_23076\71088798.py:85: RuntimeWarning: divide by zero encountered in divide
  np.abs(y_pred - y_true) / np.abs(y_true)
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages

RF:CV (Kaggle two-stage score) mean: 0.0
RF:Each split: [0. 0. 0. 0. 0.]


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\USER\AppData\Local\Temp\ipykernel_23076\71088798.py:85: RuntimeWarning: divide by zero encountered in divide
  np.abs(y_pred - y_true) / np.abs(y_true)
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages

XGB:CV (Kaggle two-stage score) mean: 0.0
XGB:Each split: [0. 0. 0. 0. 0.]


C:\Users\USER\AppData\Local\Temp\ipykernel_23076\71088798.py:85: RuntimeWarning: divide by zero encountered in divide
  np.abs(y_pred - y_true) / np.abs(y_true)


In [36]:
# 1. 全 NaN
nan_cols = X.columns[X.isna().all()]

# 2. 幾乎全 NaN（例如超過 95% 缺失）
mostly_nan_cols = X.columns[X.isna().mean() > 0.95]

# 3. 常數或接近常數欄
const_cols = X.columns[X.nunique(dropna=True) <= 1]

# 設定一個閾值，例如：非 NaN 比例 < 5% 就刪掉
low_info_cols = X.columns[(X.notna().mean() < 0.05)]

drop_cols = nan_cols.union(mostly_nan_cols).union(const_cols).union(low_info_cols)
print("要丟掉的欄位數:", len(drop_cols))

X=X.drop(columns=drop_cols)

 

要丟掉的欄位數: 13


In [37]:
print("y==0 數量:", (y == 0).sum(), "/", len(y))


y==0 數量: 65160 / 450300


In [38]:
# ===== 修正版兩階段 MAPE (處理 y_true=0) =====
def two_stage_mape_score(y_true, y_pred, epsilon=1e-6):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)

    # 避免除以零：分母加 epsilon
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)

    n = len(ape)
    frac_big = np.mean(ape > 1)  # >100% 的比例

    # 第一階段：若超過 30% 的樣本 APE > 1 → 直接 0 分
    if frac_big > 0.3:
        return 0.0

    # 第二階段：只算 APE <= 1 的樣本
    mask_ok = (ape <= 1) & np.isfinite(ape)
    if mask_ok.sum() == 0:
        return 0.0

    mape_ok = ape[mask_ok].mean()
    frac_ok = mask_ok.sum() / n
    scaled_mape = mape_ok / frac_ok
    score = 1.0 - scaled_mape

    return float(np.clip(score, 0.0, 1.0))

# ===== 包成 sklearn scorer (越大越好) =====
kaggle_scorer = make_scorer(two_stage_mape_score, greater_is_better=True)


num_cols=X.select_dtypes(include=['number']).columns.tolist()
cat_cols=X.select_dtypes(exclude=['number']).columns.tolist()

pre_linear=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
        ]),num_cols),
    ]
)

pre_tree=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
        ]),num_cols),
    ]
)


linear=LinearRegression()
pipe_linear=Pipeline([
    ('pre_linear',pre_linear),
    ('linear',linear),
])

rf=RandomForestRegressor(
    n_estimators=100,        
    max_depth=12,            
    min_samples_split=20,    
    min_samples_leaf=5,      
    max_features='sqrt',   
    bootstrap=True,          
    max_samples=0.7,         
    n_jobs=-1,
    random_state=42,
)
pipe_rf=Pipeline([
    ('pre_rf',pre_tree),
    ('rf',rf),
])

xgb=XGBRegressor(
    n_estimators=200,        
    max_depth=6,             
    learning_rate=0.1,      
    subsample=0.8,           
    colsample_bytree=0.8,    
    min_child_weight=5,      
    reg_lambda=1.0,          
    n_jobs=-1,              
    random_state=42,
    tree_method="hist"        
)
pipe_xgb=Pipeline([
    ('pre_xgb',pre_tree),
    ('xgb',xgb),
])

tscv=TimeSeriesSplit(n_splits=5)

cv_scores_linear=cross_val_score(
    pipe_linear,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("Linear:CV (Kaggle two-stage score) mean:", cv_scores_linear.mean())
print("Linear:Each split:", cv_scores_linear)

cv_scores_rf=cross_val_score(
    pipe_rf,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("RF:CV (Kaggle two-stage score) mean:", cv_scores_rf.mean())
print("RF:Each split:", cv_scores_rf)

cv_scores_xgb=cross_val_score(
    pipe_xgb,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("XGB:CV (Kaggle two-stage score) mean:", cv_scores_xgb.mean())
print("XGB:Each split:", cv_scores_xgb)

Linear:CV (Kaggle two-stage score) mean: 0.0
Linear:Each split: [0. 0. 0. 0. 0.]
RF:CV (Kaggle two-stage score) mean: 0.0
RF:Each split: [0. 0. 0. 0. 0.]
XGB:CV (Kaggle two-stage score) mean: 0.0
XGB:Each split: [0. 0. 0. 0. 0.]


In [39]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
# 假設：X(含 month, sector, y_col)、features、pipeline 都已經準備好

def frac_big_ape(y_true, y_pred, eps=1e-6):
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), eps)
    return float((ape > 1).mean())

tscv = TimeSeriesSplit(n_splits=5)
ratios = []
for tr_idx, te_idx in tscv.split(X):
    Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
    ytr, yte = y.iloc[tr_idx], y.iloc[te_idx]
    mdl = pipe_linear.fit(Xtr[num_cols], ytr)
    pred = mdl.predict(Xte[num_cols])
    pred = np.clip(pred, 0.0, None)  # y≥0
    ratios.append(frac_big_ape(yte.values, pred))

print("各折 >100% 比例：", ratios, "平均：", np.mean(ratios))


各折 >100% 比例： [0.22371752165223185, 0.34900732844770155, 0.37534976682211857, 0.31491005996002663, 0.3866222518321119] 平均： 0.32992138574283814


In [40]:
def frac_big_ape(y_true, y_pred, eps=1e-6):
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), eps)
    return float((ape > 1).mean())

tscv = TimeSeriesSplit(n_splits=5)
ratios = []
for tr_idx, te_idx in tscv.split(X):
    Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
    ytr, yte = y.iloc[tr_idx], y.iloc[te_idx]
    mdl = pipe_rf.fit(Xtr[num_cols], ytr)
    pred = mdl.predict(Xte[num_cols])
    pred = np.clip(pred, 0.0, None)  # y≥0
    ratios.append(frac_big_ape(yte.values, pred))

print("各折 >100% 比例：", ratios, "平均：", np.mean(ratios))

各折 >100% 比例： [0.3487008660892738, 0.40924716855429716, 0.33557628247834775, 0.45169886742171883, 0.4340972684876749] 平均： 0.3958640906062625


In [41]:
def frac_big_ape(y_true, y_pred, eps=1e-6):
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), eps)
    return float((ape > 1).mean())

tscv = TimeSeriesSplit(n_splits=5)
ratios = []
for tr_idx, te_idx in tscv.split(X):
    Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
    ytr, yte = y.iloc[tr_idx], y.iloc[te_idx]
    mdl = pipe_xgb.fit(Xtr[num_cols], ytr)
    pred = mdl.predict(Xte[num_cols])
    pred = np.clip(pred, 0.0, None)  # y≥0
    ratios.append(frac_big_ape(yte.values, pred))

print("各折 >100% 比例：", ratios, "平均：", np.mean(ratios))

各折 >100% 比例： [0.4108061292471685, 0.34739506995336444, 0.3012658227848101, 0.39949367088607596, 0.3853431045969354] 平均： 0.36886075949367086


In [42]:
from sklearn.dummy import DummyRegressor

dummy_reg=DummyRegressor(strategy='median')
pipe_dummy_reg=Pipeline([
    ('pre_dummy_reg',pre_tree),
    ('dummy_reg',dummy_reg),
])
cv_scores_dummy_ref=cross_val_score(
    pipe_dummy_reg,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("Dummy:CV (Kaggle two-stage score) mean:", cv_scores_dummy_ref.mean())
print("Dummy:Each split:", cv_scores_dummy_ref)

Dummy:CV (Kaggle two-stage score) mean: 0.0
Dummy:Each split: [0. 0. 0. 0. 0.]


In [43]:
df["year_month"] = pd.to_datetime(df["month"]).dt.to_period("M").dt.to_timestamp()
df = df.sort_values(["sector", "year_month"]).reset_index(drop=True)

X = df.drop("amount_new_house_transactions", axis=1)
y = df["amount_new_house_transactions"]



In [44]:
date_col  = "year_month"   
group_col = "sector"

g = df.groupby(group_col)["amount_new_house_transactions"]


#建立天真基線
preds = {
    "lag1":  g.shift(1),
    "lag12": g.shift(12),
    "ma3":   g.shift(1).rolling(3).mean(),
    "ma6":   g.shift(1).rolling(6).mean(),
}

#評估
rows = []
y_true = y.to_numpy(float)
for name, p in preds.items():
    y_pred = np.where(np.isnan(p), 0.0, p)  # 前幾期沒有歷史 → 補0
    y_pred = np.clip(y_pred, 0.0, None)     # y≥0
    score  = two_stage_mape_score(y_true, y_pred)
    ratio  = frac_big_ape(y_true, y_pred)
    rows.append((name, score, ratio))

print(pd.DataFrame(rows, columns=["baseline","two_stage_score","frac_APE>100%"])
        .sort_values("two_stage_score", ascending=False))

  baseline  two_stage_score  frac_APE>100%
0     lag1         0.995663       0.002385
2      ma3         0.989868       0.005241
3      ma6         0.981318       0.009460
1    lag12         0.945110       0.028621


In [45]:
date_col  = "year_month"   
group_col = "sector"


#時間序列排序
g = df.groupby(group_col)["amount_new_house_transactions"]

df['lag1'] = g.shift(1)
df['lag3'] = g.shift(3)
df['lag12'] = g.shift(12)
df['ma3'] = g.shift(1).rolling(3).mean()
df['ma6'] = g.shift(1).rolling(6).mean()

m=pd.to_datetime(df['year_month']).dt.month
df['month_sin']=np.sin(2*np.pi*m/12)
df['month_cos']=np.cos(2*np.pi*m/12)

#重新切分
X=df.drop('amount_new_house_transactions',axis=1)
y=df['amount_new_house_transactions']


#刪除NaN值的X欄
# 1. 全 NaN
nan_cols = X.columns[X.isna().all()]

# 2. 幾乎全 NaN（例如超過 95% 缺失）
mostly_nan_cols = X.columns[X.isna().mean() > 0.95]

# 3. 常數或接近常數欄
const_cols = X.columns[X.nunique(dropna=True) <= 1]

# 設定一個閾值，例如：非 NaN 比例 < 5% 就刪掉
low_info_cols = X.columns[(X.notna().mean() < 0.05)]

drop_cols = nan_cols.union(mostly_nan_cols).union(const_cols).union(low_info_cols)

X=X.drop(columns=drop_cols)


#切分欄位
time_cols=['lag1','lag3','lag12','ma3','ma6','month_sin','month_cos']
cat_cols=X.select_dtypes(exclude=['number']).columns.tolist()
num_cols=[c for c in X.columns if c not in time_cols+cat_cols]


#建立baseline
pre_linear=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
        ]),num_cols),
        ('time',Pipeline([
            ('imputer',SimpleImputer(strategy="constant", fill_value=0))
        ]),time_cols)
    ]
)

pre_tree=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median'))
        ]),num_cols),
        ('time',Pipeline([
            ('imputer',SimpleImputer(strategy="constant", fill_value=0))
        ]),time_cols)
    ]
)


linear=LinearRegression()
pipe_linear=Pipeline([
    ('pre_linear',pre_linear),
    ('linear',linear),
])

rf=RandomForestRegressor(
    n_estimators=100,        
    max_depth=12,            
    min_samples_split=20,    
    min_samples_leaf=5,      
    max_features='sqrt',   
    bootstrap=True,          
    max_samples=0.7,         
    n_jobs=-1,
    random_state=42,
)
pipe_rf=Pipeline([
    ('pre_rf',pre_tree),
    ('rf',rf),
])

xgb=XGBRegressor(
    n_estimators=200,        
    max_depth=6,             
    learning_rate=0.1,      
    subsample=0.8,           
    colsample_bytree=0.8,    
    min_child_weight=5,      
    reg_lambda=1.0,          
    n_jobs=-1,              
    random_state=42,
    tree_method="hist"        
)
pipe_xgb=Pipeline([
    ('pre_xgb',pre_tree),
    ('xgb',xgb),
])


tscv=TimeSeriesSplit(n_splits=5)

cv_scores_linear=cross_val_score(
    pipe_linear,X[num_cols+time_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("Linear:CV (Kaggle two-stage score) mean:", cv_scores_linear.mean())
print("Linear:Each split:", cv_scores_linear)

cv_scores_rf=cross_val_score(
    pipe_rf,X[num_cols+time_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("RF:CV (Kaggle two-stage score) mean:", cv_scores_rf.mean())
print("RF:Each split:", cv_scores_rf)

cv_scores_xgb=cross_val_score(
    pipe_xgb,X[num_cols+time_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("XGB:CV (Kaggle two-stage score) mean:", cv_scores_xgb.mean())
print("XGB:Each split:", cv_scores_xgb)


Linear:CV (Kaggle two-stage score) mean: 0.8912141569386278
Linear:Each split: [0.95593319 0.91629137 0.88740826 0.829872   0.86656597]
RF:CV (Kaggle two-stage score) mean: 0.2979890502873241
RF:Each split: [0.74058429 0.         0.74936097 0.         0.        ]
XGB:CV (Kaggle two-stage score) mean: 0.7308548434610529
XGB:Each split: [0.88271885 0.         0.93597849 0.90699291 0.92858396]


In [46]:
#刪除NaN值的X欄
# 1. 全 NaN
nan_cols = df.columns[df.isna().all()]

# 2. 幾乎全 NaN（例如超過 95% 缺失）
mostly_nan_cols = df.columns[df.isna().mean() > 0.95]

# 3. 常數或接近常數欄
const_cols = df.columns[df.nunique(dropna=True) <= 1]

# 設定一個閾值，例如：非 NaN 比例 < 5% 就刪掉
low_info_cols = df.columns[(df.notna().mean() < 0.05)]

drop_cols = nan_cols.union(mostly_nan_cols).union(const_cols).union(low_info_cols)

df=df.drop(columns=drop_cols)


# === 策略 A：丟掉暖機段 ===
def drop_warmup(df, group_col, date_col, max_lag=12):
    df = df.sort_values([group_col, date_col]).copy()
    pos = df.groupby(group_col).cumcount()
    mask = pos >= max_lag
    return df.loc[mask].reset_index(drop=True)


# === 策略 B：補開頭 NaN ===
def fill_warmup(df, group_col, lag_cols, method="bfill"):
    """
    補 lag/rolling 產生的 NaN（適合短序列）
    method 可選: "bfill" 或 "zero"
    """
    df = df.copy()
    for c in lag_cols:
        if method == "bfill":
            df[c] = df.groupby(group_col)[c].bfill()
        elif method == "zero":
            df[c] = df[c].fillna(0.0)
    return df


group_col = "sector"
date_col = "year_month"
lag_cols = ["lag1","lag3","lag12","ma3","ma6"]

# 丟掉暖機段
df_A = drop_warmup(df, group_col="sector", date_col="year_month", max_lag=12)

# 補開頭 NaN（用 bfill）
df_B = fill_warmup(df, group_col="sector", lag_cols=lag_cols, method="bfill")



In [47]:
#重新切分
X_A=df_A.drop('amount_new_house_transactions',axis=1)
y_A=df_A['amount_new_house_transactions']

#切分欄位
time_cols=['lag1','lag3','lag12','ma3','ma6','month_sin','month_cos']
cat_cols=X_A.select_dtypes(exclude=['number']).columns.tolist()
num_cols=[c for c in X_A.columns if c not in time_cols+cat_cols]


#建立baseline
pre_linear=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
        ]),num_cols),
    ],
    remainder="passthrough",
)

pre_tree=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median'))
        ]),num_cols),
    ],
    remainder="passthrough",
)


linear=LinearRegression()
pipe_linear=Pipeline([
    ('pre_linear',pre_linear),
    ('linear',linear),
])

rf=RandomForestRegressor(
    n_estimators=100,        
    max_depth=12,            
    min_samples_split=20,    
    min_samples_leaf=5,      
    max_features='sqrt',   
    bootstrap=True,          
    max_samples=0.7,         
    n_jobs=-1,
    random_state=42,
)
pipe_rf=Pipeline([
    ('pre_rf',pre_tree),
    ('rf',rf),
])

xgb=XGBRegressor(
    n_estimators=200,        
    max_depth=6,             
    learning_rate=0.1,      
    subsample=0.8,           
    colsample_bytree=0.8,    
    min_child_weight=5,      
    reg_lambda=1.0,          
    n_jobs=-1,              
    random_state=42,
    tree_method="hist"        
)
pipe_xgb=Pipeline([
    ('pre_xgb',pre_tree),
    ('xgb',xgb),
])


tscv=TimeSeriesSplit(n_splits=5)

cv_scores_linear=cross_val_score(
    pipe_linear,X_A[num_cols+time_cols],y_A,cv=tscv,scoring=kaggle_scorer
)
print("Linear:CV (Kaggle two-stage score) mean:", cv_scores_linear.mean())
print("Linear:Each split:", cv_scores_linear)

cv_scores_rf=cross_val_score(
    pipe_rf,X_A[num_cols+time_cols],y_A,cv=tscv,scoring=kaggle_scorer
)
print("RF:CV (Kaggle two-stage score) mean:", cv_scores_rf.mean())
print("RF:Each split:", cv_scores_rf)

cv_scores_xgb=cross_val_score(
    pipe_xgb,X_A[num_cols+time_cols],y_A,cv=tscv,scoring=kaggle_scorer
)
print("XGB:CV (Kaggle two-stage score) mean:", cv_scores_xgb.mean())
print("XGB:Each split:", cv_scores_xgb)


Linear:CV (Kaggle two-stage score) mean: 0.891644642268675
Linear:Each split: [0.95617168 0.91935566 0.89337745 0.82402741 0.86529101]
RF:CV (Kaggle two-stage score) mean: 0.30338740878552706
RF:Each split: [0.76028943 0.         0.75664761 0.         0.        ]
XGB:CV (Kaggle two-stage score) mean: 0.9014744529195398
XGB:Each split: [0.88633493 0.85667574 0.93163905 0.90831517 0.92440737]


In [48]:
#重新切分
X_B=df_B.drop('amount_new_house_transactions',axis=1)
y_B=df_B['amount_new_house_transactions']

#切分欄位
time_cols=['lag1','lag3','lag12','ma3','ma6','month_sin','month_cos']
cat_cols=X_B.select_dtypes(exclude=['number']).columns.tolist()
num_cols=[c for c in X_B.columns if c not in time_cols+cat_cols]


#建立baseline
pre_linear=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
        ]),num_cols),
    ],
    remainder="passthrough",
)

pre_tree=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median'))
        ]),num_cols),
    ],
    remainder="passthrough",
)


linear=LinearRegression()
pipe_linear=Pipeline([
    ('pre_linear',pre_linear),
    ('linear',linear),
])

rf=RandomForestRegressor(
    n_estimators=100,        
    max_depth=12,            
    min_samples_split=20,    
    min_samples_leaf=5,      
    max_features='sqrt',   
    bootstrap=True,          
    max_samples=0.7,         
    n_jobs=-1,
    random_state=42,
)
pipe_rf=Pipeline([
    ('pre_rf',pre_tree),
    ('rf',rf),
])

xgb=XGBRegressor(
    n_estimators=200,        
    max_depth=6,             
    learning_rate=0.1,      
    subsample=0.8,           
    colsample_bytree=0.8,    
    min_child_weight=5,      
    reg_lambda=1.0,          
    n_jobs=-1,              
    random_state=42,
    tree_method="hist"        
)
pipe_xgb=Pipeline([
    ('pre_xgb',pre_tree),
    ('xgb',xgb),
])


tscv=TimeSeriesSplit(n_splits=5)

cv_scores_linear=cross_val_score(
    pipe_linear,X_B[num_cols+time_cols],y_B,cv=tscv,scoring=kaggle_scorer
)
print("Linear:CV (Kaggle two-stage score) mean:", cv_scores_linear.mean())
print("Linear:Each split:", cv_scores_linear)

cv_scores_rf=cross_val_score(
    pipe_rf,X_B[num_cols+time_cols],y_B,cv=tscv,scoring=kaggle_scorer
)
print("RF:CV (Kaggle two-stage score) mean:", cv_scores_rf.mean())
print("RF:Each split:", cv_scores_rf)

cv_scores_xgb=cross_val_score(
    pipe_xgb,X_B[num_cols+time_cols],y_B,cv=tscv,scoring=kaggle_scorer
)
print("XGB:CV (Kaggle two-stage score) mean:", cv_scores_xgb.mean())
print("XGB:Each split:", cv_scores_xgb)

Linear:CV (Kaggle two-stage score) mean: 0.8918158741038805
Linear:Each split: [0.95625886 0.91927167 0.89287296 0.82472287 0.865953  ]
RF:CV (Kaggle two-stage score) mean: 0.2944391667188054
RF:Each split: [0.7546939  0.         0.71750194 0.         0.        ]
XGB:CV (Kaggle two-stage score) mean: 0.7244165382045891
XGB:Each split: [0.87638053 0.         0.92564204 0.89128615 0.92877396]


In [49]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform
import numpy as np

time_cols=['lag1','lag3','lag12','ma3','ma6','month_sin','month_cos']
cat_cols=X_A.select_dtypes(exclude=['number']).columns.tolist()
num_cols=[c for c in X_A.columns if c not in time_cols+cat_cols]

# 基礎模型（穩定的缺省值）
xgb_base = XGBRegressor(
    n_estimators=900,           # 偏高，讓小學習率有空間
    learning_rate=0.05,
    objective="reg:squarederror",
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",       
    n_jobs=-1,
    random_state=42,
)

pipe_xgb_ran=Pipeline([
    ('pre_xgb',pre_tree),
    ('xgb_base',xgb_base),
])

# 搜尋空間
param_dist = {
    "xgb_base__max_depth": randint(4, 7),
    "xgb_base__min_child_weight": randint(5, 21),
    "xgb_base__subsample": uniform(0.6, 0.4),
    "xgb_base__colsample_bytree": uniform(0.6, 0.4),
    "xgb_base__reg_lambda": uniform(0.3, 1.7),
    "xgb_base__learning_rate": uniform(0.03, 0.07),
}


tscv = TimeSeriesSplit(n_splits=5)

search = RandomizedSearchCV(
    pipe_xgb_ran,
    param_distributions=param_dist,
    n_iter=40,
    cv=tscv,
    scoring=kaggle_scorer,
    n_jobs=-1,
    random_state=42,
    verbose=1,
)

search.fit(X_A[num_cols+time_cols], y_A)

print("Best CV score:", search.best_score_)
print("Best params:", search.best_params_)

best_xgb = search.best_estimator_

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best CV score: 0.9313715950395031
Best params: {'xgb_base__colsample_bytree': np.float64(0.7801997007878172), 'xgb_base__learning_rate': np.float64(0.030928547281190655), 'xgb_base__max_depth': 4, 'xgb_base__min_child_weight': 16, 'xgb_base__reg_lambda': np.float64(1.2575899703374167), 'xgb_base__subsample': np.float64(0.7541666010159664)}


In [64]:
date_col  = "year_month"   # 你的日期欄
group_col = "sector"       # 群組欄
y_col     = "amount_new_house_transactions"  # 目標欄（需用到時間特徵）

df = df.copy()
df[date_col] = pd.to_datetime(df[date_col]).dt.to_period("M").dt.to_timestamp()

# 檢查欄位型別一致（常見問題：sector 有的 int、有的 str）
df[group_col] = df[group_col].astype(str)

# 哪些 key 重複？
dups = (df.groupby([group_col, date_col]).size()
          .reset_index(name="cnt")
          .query("cnt > 1")
          .sort_values(["cnt", group_col, date_col], ascending=[False, True, True]))

print("重複組合數：", len(dups))
print(dups.head(20))          # 先看看前 20 筆


重複組合數： 6365
        sector year_month  cnt
36    sector 1 2022-01-01  120
37    sector 1 2022-02-01  120
38    sector 1 2022-03-01  120
39    sector 1 2022-04-01  120
40    sector 1 2022-05-01  120
41    sector 1 2022-06-01  120
42    sector 1 2022-07-01  120
43    sector 1 2022-08-01  120
44    sector 1 2022-09-01  120
45    sector 1 2022-10-01  120
46    sector 1 2022-11-01  120
47    sector 1 2022-12-01  120
103  sector 10 2022-01-01  120
104  sector 10 2022-02-01  120
105  sector 10 2022-03-01  120
106  sector 10 2022-04-01  120
107  sector 10 2022-05-01  120
108  sector 10 2022-06-01  120
109  sector 10 2022-07-01  120
110  sector 10 2022-08-01  120


In [65]:
print(df.columns.tolist())


['month', 'sector', 'year', 'amount_new_house_transactions', 'year_end_registered_population_10k', 'total_households_10k', 'year_end_resident_population_10k', 'year_end_total_employed_population_10k', 'year_end_urban_non_private_employees_10k', 'private_individual_and_other_employees_10k', 'private_individual_ratio', 'national_year_end_total_population_10k', 'resident_registered_ratio', 'under_18_10k', '18_60_years_10k', 'over_60_years_10k', 'total', 'under_18_percent', '18_60_years_percent', 'over_60_years_percent', 'gdp_100m', 'primary_industry_100m', 'secondary_industry_100m', 'tertiary_industry_100m', 'gdp_per_capita_yuan', 'national_gdp_100m', 'national_economic_primacy', 'national_population_share', 'gdp_population_ratio', 'secondary_industry_development_gdp_share', 'tertiary_industry_development_gdp_share', 'employed_population', 'primary_industry_percent', 'secondary_industry_percent', 'tertiary_industry_percent', 'white_collar_service_vs_blue_collar_manufacturing_ratio', 'gene

In [86]:
import re
import numpy as np
import pandas as pd

def build_time_features_and_test_v5(
    df: pd.DataFrame,
    test_csv_path: str = "test/test.csv",
    sample_csv_path: str = "sample_submission.csv",
    date_col: str = "year_month",
    group_col: str = "sector",
    y_col: str = "amount_new_house_transactions",
    use_month_start: bool = True,
    use_ffill_for_y: bool = True,     # ← 新增：只用於產生時間特徵
    sum_whitelist: list | None = None,
):
    """
    v5: 統一 sector 為數字字串；每個 sector 補到 test 月份；可選擇用 ffill 的 y 來計算 lag/ma。
    回傳: feat_df, X_test_features, time_cols, sample_ids, test_id_series
    """
    df = df.copy()

    # A) sector 統一為純數字字串
    df[group_col] = (
        df[group_col].astype(str)
                      .str.replace(r"^sector\s*", "", regex=True)
                      .str.strip()
    )

    # B) 月粒度統一
    if use_month_start:
        df[date_col] = pd.to_datetime(df[date_col]).dt.to_period("M").dt.to_timestamp(how="start")
        _freq = "MS"
    else:
        df[date_col] = pd.to_datetime(df[date_col]).dt.to_period("M").dt.to_timestamp(how="end")
        _freq = "M"

    # C) 聚合字典（預設 mean，流量欄 sum）
    num_cols_all = df.select_dtypes(include=[np.number]).columns.tolist()
    agg_dict = {c: "mean" for c in num_cols_all}
    default_sum_cols = [
        "num_land_transactions","construction_area","planned_building_area","transaction_amount",
        "num_land_transactions_nearby_sectors","construction_area_nearby_sectors",
        "planned_building_area_nearby_sectors","transaction_amount_nearby_sectors",
        "num_new_house_transactions","area_new_house_transactions","amount_new_house_transactions",
        "num_new_house_transactions_nearby_sectors","area_new_house_transactions_nearby_sectors",
        "amount_new_house_transactions_nearby_sectors",
        "num_pre_owned_house_transactions","area_pre_owned_house_transactions","amount_pre_owned_house_transactions",
        "num_pre_owned_house_transactions_nearby_sectors","area_pre_owned_house_transactions_nearby_sectors",
        "amount_pre_owned_house_transactions_nearby_sectors",
    ]
    sum_whitelist = default_sum_cols if sum_whitelist is None else sum_whitelist
    pattern = re.compile(r"(transaction|transactions).*(num|area|amount)")
    for c in df.columns:
        if c in num_cols_all and pattern.search(c):
            agg_dict[c] = "sum"
    for c in sum_whitelist:
        if c in num_cols_all:
            agg_dict[c] = "sum"
    agg_dict = {k: v for k, v in agg_dict.items() if k in num_cols_all}

    df_agg = df.groupby([group_col, date_col], as_index=False).agg(agg_dict)

    # D) 解析 test → (date, sector)
    test = pd.read_csv(test_csv_path)
    tmp = test["id"].str.extract(r"^(?P<year>\d{4})\s+(?P<mon>[A-Za-z]{3})_sector\s+(?P<sector>\d+)$")
    tmp[date_col]  = pd.to_datetime(tmp["year"] + " " + tmp["mon"] + " 01")
    if not use_month_start:
        tmp[date_col] = tmp[date_col].dt.to_period("M").dt.to_timestamp(how="end")
    tmp[group_col] = tmp["sector"].astype(str).str.strip()
    target_end = tmp.groupby(group_col)[date_col].max().to_dict()

    # E) 每個 sector 補到 test 的月份
    pieces = []
    grp = df_agg[[group_col, date_col, y_col]].groupby(group_col)
    for key, g in grp:
        g2 = g.set_index(date_col).sort_index()
        start    = g2.index.min()
        end_hist = g2.index.max()
        end_test = target_end.get(str(key), end_hist)
        end      = max(end_hist, end_test)
        full_idx = pd.date_range(start=start, end=end, freq=_freq)
        g2 = g2.reindex(full_idx)
        g2.index.name = date_col
        g2[group_col] = str(key)
        pieces.append(g2.reset_index())
    hist = pd.concat(pieces, ignore_index=True)

    # F) 產生時間特徵（可選 ffill）
    if use_ffill_for_y:
        hist["y_ffill"] = hist.groupby(group_col)[y_col].ffill()
        base = hist.groupby(group_col)["y_ffill"]
    else:
        base = hist.groupby(group_col)[y_col]

    hist["lag1"]  = base.shift(1)
    hist["lag3"]  = base.shift(3)
    hist["lag12"] = base.shift(12)
    hist["ma3"]   = base.shift(1).rolling(3).mean()
    hist["ma6"]   = base.shift(1).rolling(6).mean()

    m = hist[date_col].dt.month
    hist["month_sin"] = np.sin(2*np.pi*m/12)
    hist["month_cos"] = np.cos(2*np.pi*m/12)

    time_cols = ["lag1","lag3","lag12","ma3","ma6","month_sin","month_cos"]
    feat_df   = hist[[date_col, group_col] + time_cols].copy()

    # G) 左接到 test
    X_test_idx = tmp[[date_col, group_col]].copy()
    feat_df[group_col]  = feat_df[group_col].astype(str)
    X_test_features = X_test_idx.merge(feat_df, on=[date_col, group_col], how="left")

    sample_ids = pd.read_csv(sample_csv_path)["id"].tolist()
    return feat_df, X_test_features, time_cols, sample_ids, test["id"]



def make_submission_from_best_v2(
    best_xgb,
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test_features: pd.DataFrame,
    train_features: list,
    sample_ids: list,
    out_path: str = "submission.csv",
    id_series: pd.Series | None = None,
):
    # 補齊測試欄
    miss_train = [c for c in train_features if c not in X_train.columns]
    if miss_train:
        raise ValueError(f"X_train 缺少欄位：{miss_train}")
    for c in train_features:
        if c not in X_test_features.columns:
            X_test_features[c] = np.nan

    Xtr = X_train[train_features]
    Xte = X_test_features[train_features]

    best_xgb.fit(Xtr, y_train)
    pred = best_xgb.predict(Xte)
    pred = np.clip(pred, 0.0, None)

    if id_series is None:
        sub = pd.DataFrame({"id": sample_ids, "new_house_transactions": pred})
    else:
        sub = pd.DataFrame({"id": id_series, "new_house_transactions": pred})
    sub = sub.set_index("id").reindex(sample_ids).reset_index()
    sub.to_csv(out_path, index=False)
    print(f"✅ Saved {out_path}, shape={sub.shape}")
    return sub


In [ ]:
feat_df, X_test_features, time_cols, sample_ids, test_ids = build_time_features_and_test_v5(
    df=df,
    test_csv_path="test/test.csv",
    sample_csv_path="test/sample_submission.csv",
    date_col="year_month",
    group_col="sector",
    y_col="amount_new_house_transactions",
    use_month_start=True,
    use_ffill_for_y=True,  # ← 開啟：先前向填補，再算 lag/ma
)

# 檢查 NaN 比例（應該顯著下降）
print(X_test_features[["lag1","lag3","lag12","ma3","ma6"]].isna().mean().round(3))




lag1     0.01
lag3     0.01
lag12    0.01
ma3      0.01
ma6      0.01
dtype: float64


In [93]:
pre.feature_names_in_[:20]

array(['lag1', 'lag3', 'lag12', 'ma3', 'ma6', 'month_sin', 'month_cos'],
      dtype=object)

In [94]:
X_test_full.shape

(1152, 7)

In [95]:
pre.transformers_

[('num',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['year_end_registered_population_10k',
   'total_households_10k',
   'year_end_resident_population_10k',
   'year_end_total_employed_population_10k',
   'year_end_urban_non_private_employees_10k',
   'private_individual_and_other_employees_10k',
   'private_individual_ratio',
   'national_year_end_total_population_10k',
   'resident_registered_ratio',
   'under_18_10k',
   '18_60_years_10k',
   'over_60_years_10k',
   'total',
   'under_18_percent',
   '18_60_years_percent',
   'over_60_years_percent',
   'gdp_100m',
   'primary_industry_100m',
   'secondary_industry_100m',
   'tertiary_industry_100m',
   'gdp_per_capita_yuan',
   'national_gdp_100m',
   'national_economic_primacy',
   'national_population_share',
   'gdp_population_ratio',
   'secondary_industry_development_gdp_share',
   'tertiary_industry_development_gdp_share',
   'employed_population',
   'primary_industry_percent',
   'secondary_industr

In [97]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

# 1) 取出你 RandomizedSearchCV 找到的 XGB 參數（或手動寫死）
xgb_step_name = None
for k in best_xgb.named_steps:
    if isinstance(best_xgb.named_steps[k], XGBRegressor):
        xgb_step_name = k
        break

if xgb_step_name is None:
    raise RuntimeError("找不到 XGBRegressor 步驟，請確認 best_xgb 的步驟名稱")

best_params = best_xgb.named_steps[xgb_step_name].get_params()
# 可選：刪除不必要/衝突參數
best_params.pop('n_jobs', None)
best_params.pop('random_state', None)

# 2) 只用時間特徵的前處理 + XGB
pre_time_only = ColumnTransformer(
    transformers=[
        ("time", SimpleImputer(strategy="constant", fill_value=0.0), time_cols)
    ],
    remainder="drop"
)

pipe_time_only = Pipeline([
    ("pre", pre_time_only),
    ("xgb", XGBRegressor(**best_params))
])

# 3) 用訓練資料只選 time_cols 重新 fit（很快）
X_train_time = X_A[time_cols]
pipe_time_only.fit(X_train_time, y_A)

# 4) 用測試資料的 time_cols 預測
X_test_time = X_test_features[time_cols]
y_test_pred = pipe_time_only.predict(X_test_time)
y_test_pred = np.clip(y_test_pred, 0.0, None)

# 5) 輸出提交檔（用 test.csv 的 id 順序）
submission = pd.DataFrame({
    "id": test_ids,
    "new_house_transactions": y_test_pred
})
submission.to_csv("submission.csv", index=False)
print("✅ 已輸出 submission.csv，shape =", submission.shape)


✅ 已輸出 submission.csv，shape = (1152, 2)
